# Importing Dependencies

In [66]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.layers import Dropout
import re
from nltk.corpus import stopwords
from nltk import word_tokenize
STOPWORDS = set(stopwords.words('english'))
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
from sklearn.metrics import confusion_matrix,classification_report, accuracy_score,roc_auc_score

In [2]:
#Reading the data
df = pd.read_csv("C:/Users/ayu_a/Documents/Untitled Folder/concated.csv")

In [3]:
#Removing unwanted columns
df.drop(['HotelRatings','HotelAddress','HotelName','Unnamed: 0','Authorlocation','AuthorName'],axis=1,inplace=True)

In [4]:
#Checking the nill values
(df.isnull().sum(axis=0)/len(df))*100

HotelPrice                                   0.000000
Review                                       0.000000
title                                        0.000000
Service                                      9.054203
Cleanliness                                  9.181028
Overall                                      0.000000
Value                                        9.029691
Sleep Quality                               47.947068
Rooms                                       17.773840
Location                                    22.571790
Business service (e.g., internet access)    89.300052
Check in / front desk                       82.523484
dtype: float64

In [5]:
#Droping columns having nulls with high percentage
df.drop(['Sleep Quality','Business service (e.g., internet access)','Check in / front desk'],axis=1,inplace=True)

In [6]:
df.head()

,HotelPrice,Review,title,Service,Cleanliness,Overall,Value,Rooms,Location
0,$135 - $193*,"Usually stay near the airport, but this trip w...",“Great place and location”,5.0,5.0,5.0,5.0,5.0,5.0
1,$135 - $193*,Stayed at this Hilton for 2 nights. It was lik...,"“Nice stay, nice surroundings”",4.0,4.0,4.0,4.0,4.0,5.0
2,$135 - $193*,"Stayed there one night, December 16, on the wa...",“Perfect for an overnight just off I-95”,5.0,5.0,4.0,3.0,5.0,5.0
3,$135 - $193*,I just stayed here last weekend and have alrea...,“Great Hotel”,5.0,5.0,5.0,4.0,5.0,5.0
4,$135 - $193*,My mother who is 90 and I stayed one night on ...,“good room for handicapped person”,5.0,5.0,5.0,4.0,5.0,5.0


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1476839 entries, 0 to 1476838
Data columns (total 9 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   HotelPrice   1476839 non-null  object 
 1   Review       1476839 non-null  object 
 2   title        1476839 non-null  object 
 3   Service      1343123 non-null  float64
 4   Cleanliness  1341250 non-null  float64
 5   Overall      1476839 non-null  float64
 6   Value        1343485 non-null  float64
 7   Rooms        1214348 non-null  float64
 8   Location     1143490 non-null  float64
dtypes: float64(6), object(3)
memory usage: 101.4+ MB


In [6]:
df['Overall']=df.Overall.astype('str')

In [18]:
df[df.Overall == '0.0'].index

Int64Index([284049], dtype='int64')

In [7]:
#Droping the row which has no rating.
df.drop(df[df.Overall == '0.0'].index, inplace = True) 

In [8]:
df.Overall.value_counts()

5.0    615476
4.0    463650
3.0    195121
2.0    103418
1.0     99173
Name: Overall, dtype: int64

In [9]:
df["ReviewDetails"]=df['title']+" "+df["Review"]

In [37]:
df

,HotelPrice,Review,title,Service,Cleanliness,Overall,Value,Rooms,Location,ReviewDetails
0,$135 - $193*,"Usually stay near the airport, but this trip w...",“Great place and location”,5.0,5.0,5.0,5.0,5.0,5.0,“Great place and location” Usually stay near t...
1,$135 - $193*,Stayed at this Hilton for 2 nights. It was lik...,"“Nice stay, nice surroundings”",4.0,4.0,4.0,4.0,4.0,5.0,"“Nice stay, nice surroundings” Stayed at this ..."
2,$135 - $193*,"Stayed there one night, December 16, on the wa...",“Perfect for an overnight just off I-95”,5.0,5.0,4.0,3.0,5.0,5.0,“Perfect for an overnight just off I-95” Staye...
3,$135 - $193*,I just stayed here last weekend and have alrea...,“Great Hotel”,5.0,5.0,5.0,4.0,5.0,5.0,“Great Hotel” I just stayed here last weekend ...
4,$135 - $193*,My mother who is 90 and I stayed one night on ...,“good room for handicapped person”,5.0,5.0,5.0,4.0,5.0,5.0,“good room for handicapped person” My mother w...
...,...,...,...,...,...,...,...,...,...,...
1476834,$184 - $341*,I just visited NYC this past weekend with fami...,“Awesome Location. Nice rooms!”,NaN,NaN,4.0,NaN,NaN,NaN,“Awesome Location. Nice rooms!” I just visited...
1476835,$184 - $341*,Location is alright. A good walk from Times Sq...,“its alright”,NaN,NaN,3.0,NaN,NaN,NaN,“its alright” Location is alright. A good walk...
1476836,$184 - $341*,We just got back last weekend. We were looking...,"“Fabulous for the price, nice and clean!! Perf...",3.0,3.0,4.0,4.0,3.0,NaN,"“Fabulous for the price, nice and clean!! Perf..."
1476837,$184 - $341*,Ok... while most of the reviews seem good of t...,“Nightmare!!”,NaN,NaN,1.0,NaN,NaN,NaN,“Nightmare!!” Ok... while most of the reviews ...


In [22]:
def print_plot(index):
    example = df[df.index == index][['ReviewDetails' ,'Overall','HotelPrice']].values[0]
    if len(example) > 0:
        print(example[0])
        print('rating:', example[1])
        print('price',example[2])

In [23]:
print_plot(10)

“What a great experience!” Due to unforeseen circumstances, I found myself looking for a hotel in my area. I remembered the few hotels in the White Marsh area and immediately called the Hilton. They quoted me a single king bed at $199. I then asked for military discount and the room came down to $99. Wow! What a great surprise. Especially since $199 was very high. When I got there, I definitely saw why the hotel had higher prices. WHAT A GREAT HOTEL!!!! The room had a fridge, microwave, and nice coffee area. Great sitting area with a couch and desk. And the bed! The bed was great! It molds to your body and is amazing. After a rough day, it was very nice to have such a comfortable place. I loved that they also offered free internet. If need be, I would stay here again, and I would def recommend to out of town guests. BTW, the front desk girl was very friendly and she seemed to genuinely enjoy her hotel and her job.
rating: 5.0
price $135 - $193*


In [24]:
print_plot(100)

“Hotel Monaco - great Seattle location” For location, this hotel is ideal - it is right in the heart of downtown, we walked to Pioneer Sq, Pike Street market, downtown shopping etc. The 5pm wine reception is a really nice touch. Check-in was a bit odd - they seemed not to find our booking and were a bit uncommunicative, then our room was 904, next to the elevators, which we heard constantly. Not a big deal, other then early in the morning but one to avoid. Overall, good for a short stay.
rating: 4.0
price $184 - $345*


### Removing stop words

In [10]:
df = df.reset_index(drop=True)
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
    text = BAD_SYMBOLS_RE.sub('', text) # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing. 
    #text = text.replace('x', '')
#    text = re.sub(r'\W+', '', text)
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove stopwors from text
    return text
df['ReviewDetails'] = df['ReviewDetails'].apply(clean_text)

In [11]:
df['ReviewDetails'] = df['ReviewDetails'].str.replace('\d+', '')


In [27]:
print_plot(1)

nice stay nice surroundings stayed hilton  nights like hilton garden inns nice people pleasant cheap good breakfast  small nice workout facilities pool good rooms bathrooms appreciated free shuttle mall plenty restaurants ikea etc
rating: 4.0
price $135 - $193*


In [28]:
print_plot(100)

hotel monaco great seattle location location hotel ideal right heart downtown walked pioneer sq pike street market downtown shopping etc pm wine reception really nice touch checkin bit odd seemed find booking bit uncommunicative room  next elevators heard constantly big deal early morning one avoid overall good short stay
rating: 4.0
price $184 - $345*


In [12]:
train_data = df['ReviewDetails']+" "+df['HotelPrice']

### one-hot encoding

In [13]:
#Convert categorical variable into dummy/indicator variables.
Y = pd.get_dummies(df['Overall']).values
print('Shape of label tensor:', Y.shape)

Shape of label tensor: (1476838, 5)


### Split into train and test

In [14]:
X_train, X_test, Y_train, Y_test = train_test_split(train_data,Y, test_size = 0.10, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(1329154,) (1329154, 5)
(147684,) (147684, 5)


### Tokenizing

In [15]:
# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 50000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 250
# This is fixed.
EMBEDDING_DIM = 100

tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(X_train)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 527674 unique tokens.


In [16]:
#Tokenize on test Set
tokenizer.fit_on_texts(X_test)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 553548 unique tokens.


In [17]:
# Transform each text in texts in a sequence of integers.
X_train_tokenize = tokenizer.texts_to_sequences(X_train)
#pad_sequences is used to ensure that all sequences in a list have the same length.
X_train_final = pad_sequences(X_train_tokenize, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X_train_final.shape)

Shape of data tensor: (1329154, 250)


In [18]:
#Test Set
# Transform each text in texts in a sequence of integers.
X_test_tokenize = tokenizer.texts_to_sequences(X_test)
#pad_sequences is used to ensure that all sequences in a list have the same length.
X_test_final = pad_sequences(X_test_tokenize, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor(test Set):', X_test_final.shape)


Shape of data tensor(test Set): (147684, 250)


In [43]:
print(X_train_final.shape,Y_train.shape)
print(X_test_final.shape,Y_test.shape)

(1329154, 250) (1329154, 5)
(147684, 250) (147684, 5)


# Training and Batch Processing

In [19]:
model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X_train_final.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2,return_sequences=True))
model.add(LSTM(50, dropout=0.2, recurrent_dropout=0.2,return_sequences=True))
model.add(LSTM(10,dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(5, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 250, 100)          5000000   
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 250, 100)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 250, 100)          80400     
_________________________________________________________________
lstm_2 (LSTM)                (None, 250, 50)           30200     
_________________________________________________________________
lstm_3 (LSTM)                (None, 10)                2440      
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 55        
Total params: 5,113,095
Trainable params: 5,113,095
Non-trainable params: 0
____________________________________________

In [21]:
epochs = 5
batch_size = 128

history = model.fit(X_train_final, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

Train on 1196238 samples, validate on 132916 samples
Epoch 1/5
1196238/1196238 [==============================] - 11347s 9ms/step - loss: 0.7852 - accuracy: 0.6533 - val_loss: 0.7084 - val_accuracy: 0.6879
Epoch 2/5
1196238/1196238 [==============================] - 9577s 8ms/step - loss: 0.6938 - accuracy: 0.6956 - val_loss: 0.6811 - val_accuracy: 0.7002
Epoch 3/5
1196238/1196238 [==============================] - 9620s 8ms/step - loss: 0.6589 - accuracy: 0.7132 - val_loss: 0.6680 - val_accuracy: 0.7088
Epoch 4/5
1196238/1196238 [==============================] - 9555s 8ms/step - loss: 0.6301 - accuracy: 0.7289 - val_loss: 0.6630 - val_accuracy: 0.7155
Epoch 5/5
1196238/1196238 [==============================] - 11270s 9ms/step - loss: 0.6048 - accuracy: 0.7426 - val_loss: 0.6623 - val_accuracy: 0.7182


In [23]:
accr = model.evaluate(X_test_final,Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

147684/147684 [==============================] - 2412s 16ms/step
Test set
  Loss: 0.658
  Accuracy: 0.720


In [26]:
pred_test = model.predict(X_test_final)

# Model Evaluation

In [37]:
#Confusion Matrix
Confusion_matrix = confusion_matrix(Y_test.argmax(axis=1), pred_test.argmax(axis=1))
Confusion_matrix

array([[ 8030,  1526,   141,    46,    38],
       [ 2212,  5633,  2007,   323,    61],
       [  384,  3124, 10522,  5180,   345],
       [   64,   317,  3248, 30518, 12188],
       [   45,    73,   222,  9800, 51637]], dtype=int64)

In [50]:
accuracy = accuracy_score(Y_test.argmax(axis=1), pred_test.argmax(axis=1))
accuracy

0.7200509195308903

In [51]:
print(classification_report(Y_test.argmax(axis=1), pred_test.argmax(axis=1)))

              precision    recall  f1-score   support

           0       0.75      0.82      0.78      9781
           1       0.53      0.55      0.54     10236
           2       0.65      0.54      0.59     19555
           3       0.67      0.66      0.66     46335
           4       0.80      0.84      0.82     61777

    accuracy                           0.72    147684
   macro avg       0.68      0.68      0.68    147684
weighted avg       0.72      0.72      0.72    147684



In [65]:
#Roc score
score= roc_auc_score(Y_test, pred_test,multi_class='ovo')
score

0.9283904326463178

# Test with a new review.

In [46]:
new_review = ['The room was dump; there was a small wardrobe which we could not use as it was not pleasant (to me). I will have rather place a open wardrobe.']
seq = tokenizer.texts_to_sequences(new_review)
padded = pad_sequences(seq, maxlen=MAX_SEQUENCE_LENGTH)
pred = model.predict(padded)
labels = [1,2,3,4,5]
print("Rating for the given review:",labels[np.argmax(pred)])

Rating for the given review: 1
